# Experiment tracking with MLflow

In this notebook we download a data file from the [NYC TLC trip record data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page) and we train a model to predict the duration of a taxi ride.

We add tracking to our experiment by using MLflow.

## Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

2023/05/23 00:26:50 INFO mlflow.tracking.fluent: Experiment with name 'nyc_taxi_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1', creation_time=1684794410751, experiment_id='1', last_update_time=1684794410751, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

## Download data from website

As an example, we will use Green Taxi data from January 2021 for training, and from February 2021 for validation.

In [3]:
!wget -NP ./data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
!wget -NP ./data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet

--2023-05-23 00:34:29--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
Resolviendo d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.67.246.167, 18.67.246.186, 18.67.246.47, ...
Conectando con d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)[18.67.246.167]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 1333519 (1,3M) [binary/octet-stream]
Guardando como: “./data/green_tripdata_2021-01.parquet”

green_tripdata_2021 100%[===================>]   1,27M  --.-KB/s    en 0,1s    

2023-05-23 00:34:30 (9,38 MB/s) - “./data/green_tripdata_2021-01.parquet” guardado [1333519/1333519]

--2023-05-23 00:34:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet
Resolviendo d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.67.246.167, 18.67.246.176, 18.67.246.186, ...
Conectando con d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)[18.67.246.167]

## Read and preprocess data

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    df.loc[:, categorical] = df.loc[:, categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [6]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

## Set target variable

Next we create an array with the values of our target variable.

In [7]:
target = "duration"

y_train = df_train[target].values
y_val = df_val[target].values

## Experiment tracking

In [9]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "sergiogrz")
    
    mlflow.log_param("train_data_file", "green_tripdata_2021-01.parquet")
    mlflow.log_param("valid_data_file", "green_tripdata_2021-02.parquet")
    
    alpha = 0.6
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

## Hyperparameter tuning

In [10]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import xgboost as xgb

### Defining the objective function

In [11]:
#assuming we already have the dataframes in memory, we create the matrices for xgboost
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

#params contains the hyperparameters for xgboost for a specific run
def objective(params):

    with mlflow.start_run():

        #set a tag for easier classification and log the hyperparameters
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        #model definition and training
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        #predicting with the validation set
        y_pred = booster.predict(valid)

        #rmse metric and logging
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    #we return a dict with the metric and the OK signal
    return {'loss': rmse, 'status': STATUS_OK}

### Defining the search space

In [12]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # [exp(-3), exp(0)] = [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

### Minimizing the objective function

In [13]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[01:31:09] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.              
[0]	validation-rmse:14.05191                                                                                                        
[1]	validation-rmse:10.17261                                                                                                        
[2]	validation-rmse:8.20933                                                                                                         
[3]	validation-rmse:7.28021                                                                                                         
[4]	validation-rmse:6.86801                                                                                                         
[5]	validation-rmse:6.67602                                                                                                         
[6]	validation-rmse:6.59139                                          

[0]	validation-rmse:19.18916                                                                                                        
[1]	validation-rmse:17.43343                                                                                                        
[2]	validation-rmse:15.89883                                                                                                        
[3]	validation-rmse:14.55999                                                                                                        
[4]	validation-rmse:13.39704                                                                                                        
[5]	validation-rmse:12.38651                                                                                                        
[6]	validation-rmse:11.51434                                                                                                        
[7]	validation-rmse:10.75978                                         

[61]	validation-rmse:6.31126                                                                                                        
[62]	validation-rmse:6.30950                                                                                                        
[63]	validation-rmse:6.30812                                                                                                        
[64]	validation-rmse:6.30511                                                                                                        
[65]	validation-rmse:6.30364                                                                                                        
[66]	validation-rmse:6.30146                                                                                                        
[67]	validation-rmse:6.29809                                                                                                        
[68]	validation-rmse:6.29713                                         

[122]	validation-rmse:6.22683                                                                                                       
[123]	validation-rmse:6.22599                                                                                                       
[124]	validation-rmse:6.22590                                                                                                       
[125]	validation-rmse:6.22451                                                                                                       
[126]	validation-rmse:6.22343                                                                                                       
[127]	validation-rmse:6.22276                                                                                                       
[128]	validation-rmse:6.22240                                                                                                       
[129]	validation-rmse:6.22159                                        

[183]	validation-rmse:6.18754                                                                                                       
[184]	validation-rmse:6.18689                                                                                                       
[185]	validation-rmse:6.18615                                                                                                       
[186]	validation-rmse:6.18510                                                                                                       
[187]	validation-rmse:6.18485                                                                                                       
[188]	validation-rmse:6.18305                                                                                                       
[189]	validation-rmse:6.18208                                                                                                       
[190]	validation-rmse:6.18188                                        

...

[156]	validation-rmse:6.17092                                                                                                       
[157]	validation-rmse:6.17130                                                                                                       
[158]	validation-rmse:6.17065                                                                                                       
[159]	validation-rmse:6.17054                                                                                                       
[160]	validation-rmse:6.17095                                                                                                       
[161]	validation-rmse:6.17112                                                                                                       
[162]	validation-rmse:6.17137                                                                                                       
[163]	validation-rmse:6.17167                                        

[7]	validation-rmse:11.21931                                                                                                        
[8]	validation-rmse:10.54361                                                                                                        
[9]	validation-rmse:9.95399                                                                                                         
[10]	validation-rmse:9.43761                                                                                                        
[11]	validation-rmse:8.99206                                                                                                        
[12]	validation-rmse:8.60719                                                                                                        
[13]	validation-rmse:8.27416                                                                                                        
[14]	validation-rmse:7.98908                                         

[68]	validation-rmse:6.46603                                                                                                        
[69]	validation-rmse:6.46673                                                                                                        
[70]	validation-rmse:6.46833                                                                                                        
[71]	validation-rmse:6.46927                                                                                                        
[72]	validation-rmse:6.46985                                                                                                        
[73]	validation-rmse:6.47116                                                                                                        
[74]	validation-rmse:6.47262                                                                                                        
[75]	validation-rmse:6.47338                                         

[32]	validation-rmse:6.74137                                                                                                        
[33]	validation-rmse:6.70382                                                                                                        
[34]	validation-rmse:6.67037                                                                                                        
[35]	validation-rmse:6.64119                                                                                                        
[36]	validation-rmse:6.61383                                                                                                        
[37]	validation-rmse:6.58954                                                                                                        
[38]	validation-rmse:6.56850                                                                                                        
[39]	validation-rmse:6.54953                                         

[93]	validation-rmse:6.39206                                                                                                        
[94]	validation-rmse:6.39243                                                                                                        
[95]	validation-rmse:6.39257                                                                                                        
[96]	validation-rmse:6.39282                                                                                                        
[97]	validation-rmse:6.39396                                                                                                        
[98]	validation-rmse:6.39432                                                                                                        
[99]	validation-rmse:6.39420                                                                                                        
[100]	validation-rmse:6.39448                                        

### Retraining with the optimal hyperparameters and automatic logging

In [14]:
params = {
    "learning_rate": 0.1126860623846719,
    "max_depth": 11,
    "min_child_weight": 7.128461099684721,
    "objective": "reg:linear",
    "reg_alpha": 0.04429046957254972,
    "reg_lambda": 0.09902356874800584,
    "seed": 42,
    
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/05/25 02:01:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8626d75f81a24d37a74c4d2d1c1654ba', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[02:01:00] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.18916
[1]	validation-rmse:17.43343
[2]	validation-rmse:15.89883
[3]	validation-rmse:14.55999
[4]	validation-rmse:13.39704
[5]	validation-rmse:12.38651
[6]	validation-rmse:11.51434
[7]	validation-rmse:10.75978
[8]	validation-rmse:10.11576
[9]	validation-rmse:9.56565
[10]	validation-rmse:9.09455
[11]	validation-rmse:8.69152
[12]	validation-rmse:8.35168
[13]	validation-rmse:8.06299
[14]	validation-rmse:7.82050
[15]	validation-rmse:7.61407
[16]	validation-rmse:7.43876
[17]	validation-rmse:7.28958
[18]	validation-rmse:7.16372
[19]	validation-rmse:7.05627
[20]	validation-rmse:6.96427
[21]	validation-rmse:6.88679
[22]	validation-rmse:6.82021
[23]	validation-rmse:6.76296
[24]	validation-rmse:6.71292
[25]	validation-rmse:6.67016
[26]	validation-rmse:6.63300
[27]	validation-rmse:6.59970
[28]	validation-rmse:6.57240
[29]	validation-rmse:6.54777
[30]	validat

[273]	validation-rmse:6.15399
[274]	validation-rmse:6.15380
[275]	validation-rmse:6.15321
[276]	validation-rmse:6.15285
[277]	validation-rmse:6.15306
[278]	validation-rmse:6.15304
[279]	validation-rmse:6.15330
[280]	validation-rmse:6.15337
[281]	validation-rmse:6.15305
[282]	validation-rmse:6.15252
[283]	validation-rmse:6.15272
[284]	validation-rmse:6.15225
[285]	validation-rmse:6.15223
[286]	validation-rmse:6.15202
[287]	validation-rmse:6.15186
[288]	validation-rmse:6.15211
[289]	validation-rmse:6.15132
[290]	validation-rmse:6.15085
[291]	validation-rmse:6.15054
[292]	validation-rmse:6.15043
[293]	validation-rmse:6.15029
[294]	validation-rmse:6.15019
[295]	validation-rmse:6.15003
[296]	validation-rmse:6.14991
[297]	validation-rmse:6.14975
[298]	validation-rmse:6.14958
[299]	validation-rmse:6.14924
[300]	validation-rmse:6.14928
[301]	validation-rmse:6.14900
[302]	validation-rmse:6.14853
[303]	validation-rmse:6.14792
[304]	validation-rmse:6.14712
[305]	validation-rmse:6.14710
[306]	vali

2023/05/25 02:01:12 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/25 02:01:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sgrodriguez/miniconda3/envs/mlops_dtc/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
